In [4]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [7]:
import os
import numpy as np
import pandas as pd

In [2]:
with_mask_files = os.listdir('data/with_mask')
print(with_mask_files[0:5])
print(with_mask_files[-5:])

['with_mask_3643.jpg', 'with_mask_1550.jpg', 'with_mask_483.jpg', 'with_mask_1837.jpg', 'with_mask_2716.jpg']
['with_mask_2388.jpg', 'with_mask_2678.jpg', 'with_mask_781.jpg', 'with_mask_3546.jpg', 'with_mask_1197.jpg']


In [3]:
without_mask_files = os.listdir('data/without_mask')
print(without_mask_files[0:5])
print(without_mask_files[-5:])

['without_mask_2595.jpg', 'without_mask_143.jpg', 'without_mask_3805.jpg', 'without_mask_532.jpg', 'without_mask_3771.jpg']
['without_mask_97.jpg', 'without_mask_1401.jpg', 'without_mask_2133.jpg', 'without_mask_2486.jpg', 'without_mask_251.jpg']


In [5]:
with_mask_path = 'data/with_mask/'

data = []

for img_file in with_mask_files:
    image = Image.open(with_mask_path + img_file)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
np.save('with_mask.npy', data)

/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [6]:
without_mask_path = 'data/without_mask/'

data = []

for img_file in without_mask_files:
    image = Image.open(without_mask_path + img_file)
    image = image.resize((128, 128))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
np.save('without_mask.npy', data)

In [8]:
import numpy as np
import cv2
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [51]:
with_mask = np.load('with_mask.npy', allow_pickle=True)
without_mask = np.load('without_mask.npy', allow_pickle=True)

In [52]:
wm = with_mask.shape
wtm = without_mask.shape

In [53]:
wm[0]

3725

In [54]:
wtm[0]

3828

In [55]:
with_mask = with_mask.reshape(wm[0], 49152)
without_mask = without_mask.reshape(wtm[0],49152)

In [56]:
X = np.r_[with_mask, without_mask]

labels = np.zeros(X.shape[0])
labels[202:] = 1.0
names = {0: 'Mask', 1: 'No Mask'}

In [58]:
x_train.shape

(5664, 49152)

In [60]:
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [61]:
x_train[0]

array([16662.97769751,  2545.14262764,  4728.03482387])

In [62]:
x_train.shape

(5664, 3)

In [67]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.25)

In [68]:
svm = SVC()
svm.fit(x_train, y_train)

SVC()

In [69]:
# x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [70]:
accuracy_score(y_test, y_pred)

0.9698253043938592

In [3]:
haar_data = cv2.CascadeClassifier('data.xml')

In [37]:
with_mask.shape

(3725, 49152)

In [38]:
without_mask.shape

(3828, 49152)

In [ ]:
import cv2
capture = cv2.VideoCapture('mask.mp4')
data = []
haar_data = cv2.CascadeClassifier('data.xml')

while True:
    flag, img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x, y, w, h in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 4)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50, 50))
            if len(data) < 400:
                data.append(face)
        cv2.imshow('result', img)
        if cv2.waitKey(2) == 27 or len(data) >= 200:
            break
capture.release()
cv2.destroyAllWindows()